In [1]:
# Objective: Keep the dimension of zone (combination of newspaper and zone_id) and remove the dimension of target stores
# With the same exclusion cretioria: 
#   a) 1 event only - removed 
#   b) keep all the stores when it is/was open
#   c) total circ != 0
#   d) the same inclusion stores (recently open in April 2018) & circ stores are before the disappare of the weekly sales
import pandas as pd
import glob
import numpy as np
import datetime
import logging
import re
import os
import gc
today_str=str(datetime.datetime.now().date())
writer_folder="/home/jian/Projects/Big_Lots/Newspaper/zone_zip_level_output_"+today_str+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)
    
logging.basicConfig(filename='read_all_newspaper_'+today_str+'_files.log', level=logging.INFO)

In [2]:
start_running=str(datetime.datetime.now())
logging.info("Newspaper level - Start to run: "+ start_running)

folder="/home/jian/Projects/Big_Lots/Newspaper/All newspaper files for reading JL/"

file_list=glob.glob(folder+"*")
logging.info("Newspaper level - Total files: "+str(len(file_list)))

Selected_Col=['projectname','storeid','productid','zoneid','zips','totalcirc']
output_combined_all_date=pd.DataFrame()
check_df=pd.DataFrame()
finished=[]

In [3]:
recent_52_weeks_files_list=[]
for file in file_list:           
    date=file.split("/")[len(file.split("/"))-1].split(" ")[0]+" "+file.split("/")[len(file.split("/"))-1].split(" ")[1]+" "+\
    file.split("/")[len(file.split("/"))-1].split(" ")[2]
    date=date.replace("Sept","Sep")

    if len(date.split(" ")[0])==3:
        date=datetime.datetime.strptime(date,"%b %d %Y").date()
    else:
        date=datetime.datetime.strptime(date,"%B %d %Y").date()
    if date>datetime.date(2017,5,10):
    
        recent_52_weeks_files_list=recent_52_weeks_files_list+[file]
file_list=recent_52_weeks_files_list
# Update the file list into the most recent 52 weeks events

In [4]:
zip_circ_analysis=pd.read_csv("/home/jian/Projects/Big_Lots/Newspaper/Analysis/zip_level_information.csv")
zip_exclude_1=zip_circ_analysis[(zip_circ_analysis['zip_cd']==0) | (zip_circ_analysis['zip_cd']==99999)]
zip_exclude_2=zip_circ_analysis[(zip_circ_analysis['Event_Count']==1) | (zip_circ_analysis['total_circ']==0)]
zip_exclude=zip_exclude_1.append(zip_exclude_2)[['zip_cd','total_circ']]

In [6]:
len(zip_exclude['zip_cd'].unique())

5135

In [7]:
zip_circ_analysis[zip_circ_analysis['zip_cd'].isin(zip_exclude['zip_cd'].unique().tolist())].head(2)

,zip_cd,total_circ,Event_Count,Circ per Event,cost,loyalty_pctg_zip_sales,HH15,total_pop,F_25to54,Circ Penetration of F25_54,...,2018-02-11,2018-02-18,2018-02-25,2018-03-04,2018-03-18,2018-03-25,2018-04-08,2018-04-15,2018-04-22,2018-04-29
0,0,1.030379e+06,30,34345.966667,54341.108850,0.513134,NaN,NaN,NaN,NaN,...,53072.0,53072.0,53072.0,53072.0,53072.0,53072.0,53072.0,53072.0,53072.0,53072.0
2,1002,1.671939e+03,1,1671.938507,108.124263,0.515322,9671.2,29970.0,3840.0,0.435401,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
sales_data=pd.read_excel("/home/jian/BiglotsCode/outputs/Output_2018-05-05/wide_sales_date2018-05-05.xlsx")
# Any one week sales >0 in the most recent month, it indicated an "Open" status up to 2018-05-05
recent_month=sales_data.columns.tolist()[len(sales_data.columns.tolist())-4:len(sales_data.columns.tolist())]
most_recent_open_stores=sales_data[['location_id']+recent_month]
most_recent_open_stores['4_week_total_sales']=most_recent_open_stores[most_recent_open_stores.columns[1]]+most_recent_open_stores[most_recent_open_stores.columns[2]]+\
                                                                  most_recent_open_stores[most_recent_open_stores.columns[3]]+most_recent_open_stores[most_recent_open_stores.columns[4]]
                                                 
most_recent_open_stores=most_recent_open_stores[most_recent_open_stores['4_week_total_sales']>0]
recent_open_stores=most_recent_open_stores['location_id'].unique().tolist()
recent_open_stores=[str(x) for x in recent_open_stores]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [65]:
store_latest_non_zero_week=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2018-05-12.csv")
store_latest_non_zero_week=store_latest_non_zero_week[['location_id','week_end_date','sales']]
store_latest_non_zero_week['week_end_date']=store_latest_non_zero_week['week_end_date'].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").date())
store_latest_non_zero_week=store_latest_non_zero_week[store_latest_non_zero_week['sales']!=0]
store_latest_non_zero_week=store_latest_non_zero_week.sort_values(["location_id",'week_end_date'],ascending=[True,False]).drop_duplicates(['location_id'])
del store_latest_non_zero_week['sales']
store_latest_non_zero_week.columns=['storeid','latest_date']
store_latest_non_zero_week['storeid']=store_latest_non_zero_week['storeid'].astype(str)
store_latest_non_zero_week.reset_index(inplace=True)
store_latest_non_zero_week['storeid']=store_latest_non_zero_week['storeid'].astype(int)

del store_latest_non_zero_week['index']
store_latest_non_zero_week['storeid']=store_latest_non_zero_week['storeid'].astype(str)

In [66]:
logging.basicConfig(filename='read_all_newspaper_with_zone_zip_level'+today_str+'_files.log', level=logging.INFO)

In [67]:
for file in file_list:
    date=file.split("/")[len(file.split("/"))-1].split(" ")[0]+" "+file.split("/")[len(file.split("/"))-1].split(" ")[1]+" "+\
    file.split("/")[len(file.split("/"))-1].split(" ")[2]
    date=date.replace("Sept","Sep")

    if len(date.split(" ")[0])==3:
        date=datetime.datetime.strptime(date,"%b %d %Y").date()
    else:
        date=datetime.datetime.strptime(date,"%B %d %Y").date()
        
    # if date>=datetime.datetime(2017,4,1).date(): #Most recent 12 months to reduce stores without information

    df=pd.read_excel(file,sheetname="to_read",dtype=str,na_values=['NULL','null','Null',""," "])
    df=df[Selected_Col]
    df['zips']=df['zips'].fillna('null')
    df['zips']=df['zips'].replace('nan','null')
    df['totalcirc']=df['totalcirc'].astype(float)

    df['Date']=date
    df=df[df['storeid'].isin(recent_open_stores)]
    df=pd.merge(df,store_latest_non_zero_week,on="storeid",how="left")
    df=df[df['latest_date']>=date]
    del df['latest_date']
    # check_df=check_df.append(df)



    df_combine_date=pd.DataFrame()
    #Use only product id to adjust null
    for productid,group in df.groupby(['productid']):
        # df_product_store = group.copy()
        df_product_store_zone_NA=group[group['zips']=='null']
        df_product_store_zone_NA.reset_index(inplace=True)
        del df_product_store_zone_NA['index']
        
        df_product_store_zone_zip=group[group['zips']!='null']
        df_product_store_zone_zip.reset_index(inplace=True)
        del df_product_store_zone_zip['index']        

        if len(df_product_store_zone_NA)==0:
            df_product_store_zone_zip=group.groupby(['storeid','productid','zoneid','zips'])['totalcirc'].sum().to_frame()
            df_product_store_zone_zip['circ_proportion_of_Null']=0
            df_product_store_zone_zip['adjusted_circ_about_Null']=df_product_store_zone_zip['totalcirc']
            df_product_store_zone_zip.reset_index(inplace=True)
            #V del df_product_store_zone_zip['index']

        elif len(df_product_store_zone_zip)==0:              
            df_product_store_zone_NA['extract_from_zone 5 digit']=df_product_store_zone_NA['zoneid'].apply(lambda x: re.findall(r"\b\d{5}\b", x))
            unique_len_from_zone=len(df_product_store_zone_NA['extract_from_zone 5 digit'].apply(lambda x :len(x)).unique())
            if unique_len_from_zone==0:
                df_product_store_zone_zip=group.groupby(['storeid','productid','zoneid','zips'])['totalcirc'].sum().to_frame()
                df_product_store_zone_zip['circ_proportion_of_Null']=1
                df_product_store_zone_zip['adjusted_circ_about_Null']=df_product_store_zone_zip['totalcirc']
                df_product_store_zone_zip.reset_index(inplace=True)
                # del df_product_store_zone_zip['index']
            else:                                                                                                    
                group['extract_from_zone 5 digit']=group['zoneid'].apply(lambda x: re.findall(r"\b\d{5}\b", x))                                                                        
                group['zips']=group['extract_from_zone 5 digit'].apply(lambda x: str(x)[1:(len(str(x))-1)].replace("'","").replace("'",""))
                df_product_store_zone_zip=group.groupby(['storeid','productid','zoneid','zips'])['totalcirc'].sum().to_frame()
                df_product_store_zone_zip['circ_proportion_of_Null']=1
                df_product_store_zone_zip['adjusted_circ_about_Null']=df_product_store_zone_zip['totalcirc']
                df_product_store_zone_zip.reset_index(inplace=True)
                # del df_product_store_zone_zip['index']

        elif len(df_product_store_zone_NA)>=1:
            na_zip_circ=df_product_store_zone_NA['totalcirc'].sum()
            df_product_store_zone_zip=df_product_store_zone_zip.groupby(['storeid','productid','zoneid','zips'])['totalcirc'].sum().to_frame()
            df_product_store_zone_zip.reset_index(inplace=True)
            
            df_product_store_zone_zip['circ_proportion_of_Null']=df_product_store_zone_zip['totalcirc'].apply(lambda x:x/sum(df_product_store_zone_zip['totalcirc']))
            df_product_store_zone_zip['adjusted_circ_about_Null']=df_product_store_zone_zip['totalcirc']+df_product_store_zone_zip['circ_proportion_of_Null']*na_zip_circ
        df_combine_date=df_combine_date.append(df_product_store_zone_zip)

    df_combine_date['Date']=date
    finished=finished+[str(date)]
    logging.info(str(date)+ " : Done of "+str(len(finished)))
    print(str(date)+ " : Done of "+str(len(finished)))

    output_combined_all_date=output_combined_all_date.append(df_combine_date) 
output_combined_all_date.reset_index(inplace=True)
del output_combined_all_date['index']
output_combined_all_date.to_excel(writer_folder+"BL_combined newspaper data_JL_"+today_str+".xlsx",index=False)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2018-04-29 : Done of 1
2017-06-11 : Done of 2
2018-03-04 : Done of 3
2018-04-08 : Done of 4
2017-12-03 : Done of 5
2017-10-29 : Done of 6
2018-03-18 : Done of 7
2018-03-25 : Done of 8
2017-07-30 : Done of 9
2017-12-17 : Done of 10
2017-11-12 : Done of 11
2018-04-15 : Done of 12
2017-11-22 : Done of 13
2017-10-22 : Done of 14
2017-11-26 : Done of 15
2017-11-19 : Done of 16
2017-09-03 : Done of 17
2018-01-07 : Done of 18
2017-10-08 : Done of 19
2017-11-23 : Done of 20
2018-04-22 : Done of 21
2018-02-11 : Done of 22
2017-05-21 : Done of 23
2017-12-10 : Done of 24
2017-11-05 : Done of 25
2017-07-16 : Done of 26
2017-08-27 : Done of 27
2017-09-24 : Done of 28
2018-02-25 : Done of 29
2018-02-18 : Done of 30
2017-06-25 : Done of 31
2017-05-28 : Done of 32


In [68]:
household=pd.read_csv("/home/jian/Docs/Household_and_Population/2016/ZipcodePopulation_FromSP.csv",dtype=str)
household=household[['ZIP_CODE',"HH15"]]
household.columns=['zip_cd','Households']
household['Households']=household['Households'].fillna(0)
household['Households']=household['Households'].astype(float)
gc.collect()

2296

In [71]:
output_combined_all_date['Date_Str']=output_combined_all_date['Date'].apply(lambda x: str(x))
output_combined_all_date['key']="S_"+output_combined_all_date['storeid']+" | P_"+output_combined_all_date['productid']+" | Z"+\
                                output_combined_all_date['zoneid']+" | "+output_combined_all_date['Date_Str']
del output_combined_all_date['Date_Str']

output_combined_all_date['zip_list']=output_combined_all_date['zips'].apply(lambda x: [ zip_str.strip().zfill(5) for zip_str in x.split(",")])
output_combined_all_date['zip_count']=output_combined_all_date['zip_list'].apply(lambda x: len(x))

output_combined_all_date_1=output_combined_all_date[output_combined_all_date['zip_count']==1]
output_combined_all_date_M=output_combined_all_date[output_combined_all_date['zip_count']!=1]
output_combined_all_date_1['zip_cd']=output_combined_all_date_1['zips'].apply(lambda x: str(x).zfill(5))
output_combined_all_date_1['zip_list_proportion']=1
output_combined_all_date_1['adjusted_circ_with_list']=output_combined_all_date_1['adjusted_circ_about_Null']
output_combined_all_date_M.reset_index(inplace= True)
del output_combined_all_date_M['index']
output_combined_all_date_M_agg=pd.DataFrame()

In [75]:
for i in range(len(output_combined_all_date_M)):
    df=output_combined_all_date_M.iloc[i:(i+1),:]
    df.reset_index(inplace=True)
    del df['index']
    df_to_app=pd.DataFrame()
    for zip_i in list(set(df['zip_list'][0])):
        df_1_zip=df.copy()
        df_1_zip['zip_cd']=zip_i
        df_to_app=df_to_app.append(df_1_zip)
    df_to_app=pd.merge(df_to_app,household,on="zip_cd",how="left")
    df_to_app['Households']=df_to_app['Households'].fillna(0)
    if df_to_app['Households'].sum()!=0:
        df_to_app['zip_list_proportion']=df_to_app['Households']/df_to_app['Households'].sum()
        df_to_app['adjusted_circ_with_list']=df_to_app['adjusted_circ_about_Null']*df_to_app['zip_list_proportion']
    else:
        logging.info("All 0 population: "+str(df_to_app['Date'].unique()[0])+" "+str(df_to_app['storeid'].unique()[0])+" "+str(df_to_app['productid'].unique()[0]))
        logging.info(df_to_app['zip_cd'])
        df_to_app['zip_list_proportion']=1/len(df_to_app)
        df_to_app['adjusted_circ_with_list']=df_to_app['adjusted_circ_about_Null']*df_to_app['zip_list_proportion']
        
    output_combined_all_date_M_agg=output_combined_all_date_M_agg.append(df_to_app)
    if i % 1000 == 100:
        logging.info(str(datetime.datetime.now())+" :")
        logging.info("output_combined_all_date_M_agg done of the row: "+str(i))
        
        # print(str(datetime.datetime.now())+"output_combined_all_date_M_agg done of the row: "+str(i))
output_adjusted_final=output_combined_all_date_M_agg.append(output_combined_all_date_1)
output_adjusted_final.to_csv(writer_folder+"BL_combined newspaper final detailed_JL_"+today_str+".csv",index=False)

2018-06-22 15:37:29.751944output_combined_all_date_M_agg done of the row: 100
2018-06-22 15:38:18.785722output_combined_all_date_M_agg done of the row: 1100
2018-06-22 15:39:08.043613output_combined_all_date_M_agg done of the row: 2100


KeyboardInterrupt: 

In [42]:
all_store_in_newspaper=output_adjusted_final['storeid'].unique().tolist()                                                                                                    
                                                                                                     
output_no_detail=output_adjusted_final.groupby(['zip_cd','Date'])['adjusted_circ_with_list'].sum().to_frame()
output_no_detail.reset_index(inplace=True)
output_no_detail=output_no_detail.sort_values(['Date','zip_cd'])
output_no_detail.to_csv(writer_folder+"BL_newspaper count by zip by date_JL_"+today_str+".csv",index=False)

output_no_detail=pd.read_csv(writer_folder+"BL_newspaper count by zip by date_JL_"+today_str+".csv")
test_agg_wide=output_no_detail.pivot(index="zip_cd",columns='Date',values='adjusted_circ_with_list')
test_agg_wide.reset_index(inplace=True)
test_agg_wide_columns=test_agg_wide.columns.tolist()[1:]
test_agg_wide_columns=["newspaper_"+x for x in test_agg_wide_columns]
test_agg_wide.columns=[['zip_cd']+test_agg_wide_columns]
test_agg_wide=test_agg_wide.fillna(0)
test_agg_wide.to_csv(writer_folder+"BL_newspaper wide of date by zip_JL_"+today_str+".csv",index=False)


,Date,Households,adjusted_circ_about_Null,adjusted_circ_with_list,circ_proportion_of_Null,key,productid,storeid,totalcirc,zip_cd,zip_count,zip_list,zip_list_proportion,zips,zoneid
0,2017-03-19,9779.5,2711.969187,1154.166293,0.041108,S_5223 | P_10 | ZZn_01D | 2017-03-19,10,5223,2348.0,07860,6,"[07826, 07848, 07439, 07827, 07860, 07871]",0.425582,"07826, 07848, 07439, 07827, 07860, 07871",Zn_01D
1,2017-03-19,1670.0,2711.969187,197.091642,0.041108,S_5223 | P_10 | ZZn_01D | 2017-03-19,10,5223,2348.0,07848,6,"[07826, 07848, 07439, 07827, 07860, 07871]",0.072675,"07826, 07848, 07439, 07827, 07860, 07871",Zn_01D


In [76]:
output_combined_all_date_M_agg.head(2)

,storeid,productid,zoneid,zips,totalcirc,circ_proportion_of_Null,adjusted_circ_about_Null,Date,key,zip_list,zip_count,zip_cd,Households,zip_list_proportion,adjusted_circ_with_list
0,5223,10,Zn_01D,"07439, 07822, 07826, 07827, 07848, 07851, 0786...",1637.0,0.036757,2424.918358,2018-04-29,S_5223 | P_10 | ZZn_01D | 2018-04-29,"[07439, 07822, 07826, 07827, 07848, 07851, 078...",8,07822,415.6,0.017647,42.792616
1,5223,10,Zn_01D,"07439, 07822, 07826, 07827, 07848, 07851, 0786...",1637.0,0.036757,2424.918358,2018-04-29,S_5223 | P_10 | ZZn_01D | 2018-04-29,"[07439, 07822, 07826, 07827, 07848, 07851, 078...",8,07860,9779.5,0.415253,1006.954744


In [47]:
[x for x in range(8,16)]

[8, 9, 10, 11, 12, 13, 14, 15]

In [49]:
file_list

['/home/jian/Projects/Big_Lots/Newspaper/All newspaper files for reading JL/March 19 2017 4 STD.xlsx',
 '/home/jian/Projects/Big_Lots/Newspaper/All newspaper files for reading JL/August 28 2016 4 STD.xlsx',
 '/home/jian/Projects/Big_Lots/Newspaper/All newspaper files for reading JL/April 29 2018 4 STD.xlsx',
 '/home/jian/Projects/Big_Lots/Newspaper/All newspaper files for reading JL/June 11 2017 KJ.xlsx',
 '/home/jian/Projects/Big_Lots/Newspaper/All newspaper files for reading JL/December 4 2016 4 STD and 6 STD.xlsx',
 '/home/jian/Projects/Big_Lots/Newspaper/All newspaper files for reading JL/March 4 2018 2 STD.xlsx',
 '/home/jian/Projects/Big_Lots/Newspaper/All newspaper files for reading JL/April 8 2018 4 STD.xlsx',
 '/home/jian/Projects/Big_Lots/Newspaper/All newspaper files for reading JL/December 3 2017 6 STD.xlsx',
 '/home/jian/Projects/Big_Lots/Newspaper/All newspaper files for reading JL/Sept 4 2016 4 STD.xlsx',
 '/home/jian/Projects/Big_Lots/Newspaper/All newspaper files for r